## Step 2. 가져온 뉴스 링크에서 본문 추출하기

#### 뉴스 링크를 가져와서 저장했기 때문에, 역시 해당 url을 요청하여 뉴스 본문을 가져오기 위해서는 step 1과 같은 package가 필요합니다.

In [1]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### Step 1. 에서 저장한 엑셀 파일을 import 해 줍니다.

In [2]:
data = pd.read_excel('./Data/news_info_with_link.xlsx')

In [3]:
data.head()

,제목,출처,날짜,원문링크
0,공시위반 4개 법인 등에 과징금 3억 9200만원,국민일보,2017-08-30 21:46,http://news.naver.com/main/read.nhn?mode=LS2D&...
1,"박삼구측, 금호타이어 상표권 계약서 초안에 수정의견 제시",뉴시스,2017-08-30 21:41,http://news.naver.com/main/read.nhn?mode=LS2D&...
2,금융소비자 정보포털 '파인' 개설 1주년 맞아 새 단장,전자신문,2017-08-30 20:31,http://news.naver.com/main/read.nhn?mode=LS2D&...
3,7월 기업 직접금융 조달액 14조 4168억원…전월 比 0.6% 증가,헤럴드경제,2017-08-30 19:52,http://news.naver.com/main/read.nhn?mode=LS2D&...
4,"KIC, 내년부터 주주 의결권 행사한다",파이낸셜뉴스,2017-08-30 19:32,http://news.naver.com/main/read.nhn?mode=LS2D&...


#### 첫번째 기사의 링크에 먼저 접근해서 본문을 가져와 보겠습니다.

In [4]:
url = data.loc[0,'원문링크']

#### 역시 requests.get()으로 url을 요청하고, BeaurifulSoup를 활용하여 해당 HTML페이지를 파싱해줍니다.

In [5]:
r = requests.get(url)
soup = BeautifulSoup(r.text,'lxml')

#### 페이지 구조를 확인하여 뉴스 본문이 어떤 구조 안에 포함되어있는지 알아낸 후에 soup.find()를 활용해서 가져옵니다.

In [6]:
body = soup.find('div',{'id':'articleBodyContents'}).text
body = body.split('flash_removeCallback() {}')[1]
body = body.strip() # string 양끝의 blank 및 줄바꿈을 삭제

#### 이제, 모든 기사에 대해 본문을 가져올 수 있도록 함수를 짜봅시다.

In [7]:
def getNewsBody(df):
    news_body = []
    for index, row in df.iterrows():
        url = row['원문링크']
        r = requests.get(url)
        soup = BeautifulSoup(r.text,'lxml')
        body = soup.find('div',{'id':'articleBodyContents'}).text
        body = body.split('flash_removeCallback() {}')[1]
        body = body.strip() # string 양끝의 blank 및 줄바꿈을 삭제
        
        news_body.append(body)
    df['본문'] = news_body
    df = df.drop('원문링크',axis=1)
    
    writer = pd.ExcelWriter('./Data/' + 'news_info_with_body.xlsx')
    df.to_excel(writer,'Naver Finance',index=False)
    writer.save()
    print('News body have been successfully extracted.')
    return 

In [8]:
data = pd.read_excel('./Data/news_info_with_link.xlsx')

In [9]:
getNewsBody(data)

News body have been successfully extracted.
